In [1]:
# IMPORTS

import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import tensorflow
import lightkurve as lk

from PIL import Image

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image


2022-07-04 14:45:01.252018: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 14:45:01.252046: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# PHASE AND BIN FUNCTION

def phase_and_bin(data,period,t0,bin_time):
    data_phased = data.fold(period=period,epoch_time=t0)
    data_binned = data_phased.bin(bin_time)
    return data_binned


In [3]:
# CONVERT PLOT TO ARRAY FUNCTION

def plot_to_array(pltdata):

    lc = pltdata.plot(linewidth=0,marker=".")
    img = np.asarray(bytearray(lc.read()),dtype=np.uint8)
    
    return img

In [4]:
# DOWNLOAD (& ADD TO BIG ARRAY)

def download(search,arr):
    lc = search.download()
    array = plot_to_array(lc)
    arr.append(array)
    return arr

In [5]:
# GET ARRAY OF IMAGE ARRAYS FOR A FILE

def get_lightcurves(filename):

    tbl = pd.read_csv(os.path.abspath(filename),delimiter=",",comment="#")
    
    colnames = tbl.columns.values.tolist()
    if "tid" in colnames:
        TICs = tbl["tid"].astype(str).str[4:]
    elif "tic_id" in colnames:
        TICs = tbl["tic_id"].astype(str).str[4:]
    else:
        print("No TIC ID column found.")

    x = 0

    imgarray = np.array([])

    while x < len(TICs):
        name = TICs[x]
        try:
            search = lk.search_lightcurve("TIC " + name,author="SPOC",sector=1)
            download(search,imgarray)
        except:
            print("No lightcurve found.")
    
        x = x + 1

        return imgarray

In [6]:
# FETCH DATA (TESS DATA W/ LIGHTKURVE)

# have it so user downloads csv table in certain format from NASA exoplanet archives (with the below headers), then inputs the file name

#exonames_tbl = pd.read_csv(os.path.abspath("PS_2022.06.27_08.12.38.csv"),delimiter=",",comment="#")
#TICs = exonames_tbl["tic_id"]
#periods = exonames_tbl["pl_orbper"] # for phasing
#epochs = exonames_tbl["pl_orbtper"] # for phasing

#bintime = 15/24/60

#exotrainimgs = []

#for x in range(0,17):
    #print(TICs[x])
    #name = TICs[x]
    #if np.isnan(periods[x]) == False and np.isnan(epochs[x]) == False:
        #train_exo_search = lk.search_lightcurve(str(name),author="SPOC")[0:5]
        #lc_exo_train = train_exo_search.download_all().stitch()
        
        #lc_exo_train_binned = phase_and_bin(lc_exo_train,periods[x],epochs[x],bintime)
        
        #imgarray = plot_to_array(lc_exo_train_binned)
        #exotrainimgs.append(imgarray) # more on this part next!!
        

In [9]:
# CREATE TRAIN AND TEST DATASETS

def predictExo(exotrainfile,noexotrainfile,testfile):

    train = ImageDataGenerator(rescale=1/255)
    test = ImageDataGenerator(rescale=1/255)
    batchsize = 7

    train_ds = train.flow(
        get_lightcurves(exotrainfile),
        get_lightcurves(noexotrainfile),
        batch_size = batchsize)

    test_ds = test.flow(
        get_lightcurves(testfile),
        batch_size = batchsize)

    # BUILD CNN MODEL

    model = keras.Sequential()

    model.add(keras.layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
    model.add(keras.layers.MaxPool2D(2,2))

    model.add(keras.layers.Conv2D(64,(3,3),activation="relu"))
    model.add(keras.layers.MaxPool2D(2,2))

    model.add(keras.layers.Conv2D(128,(3,3),activation="relu"))
    model.add(keras.layers.MaxPool2D(2,2))

    model.add(keras.layers.Conv2D(128,(3,3),activation="relu"))
    model.add(keras.layers.MaxPool2D(2,2))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(512,activation="relu"))

    model.add(keras.layers.Dense(1,activation="sigmoid"))

    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    # TRAIN DATA

    model.fit(
        train_ds,
        steps_per_epoch = 250,
        epochs = 10,
        validation_data = test_ds
    )

    for y in testfile:

        Y = get_lightcurves(testfile[y])
        pic = Image.fromarray(Y)
        pic.show()
        X = np.expand_dims(Y,axis=0)

        val = model.predict(X)
        print(val)

        if val == 1:
            plt.xlabel("Exoplanet detected!",fontsize=30)
        elif val == 0:
            plt.xlabel("No exoplanet detected.",fontsize=30)

In [10]:
predictExo("PS_2022.06.27_08.12.38.csv","TOI_2022.06.29_08.07.35.csv","PS_2022.07.04_02.32.20.csv")

No lightcurve found.
No lightcurve found.


/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/lightkurve/search.py:341: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(
/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/lightkurve/search.py:341: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(


IndexError: list index out of range